In [1]:
import warnings
#warnings.filterwarnings('ignore')
import scrapy
import logging
from scrapy.crawler import CrawlerProcess
from scrapy import signals
import csv

#logging.getLogger('scrapy').propagate = False
class CatalogSpider(scrapy.Spider):

    name = "catalogdata"
    start_urls = ["http://dmoztools.net/Arts/Animation/Anime/Titles/3/3%C3%973_Eyes/",
                  "http://dmoztools.net/Computers/Software/Databases/Data_Warehousing/Consultants/",
                  "http://dmoztools.net/Health/Animal/Drugs_and_Medications/",
                  "http://dmoztools.net/Health/Animal/Veterinary_Medicine/",
                  "http://dmoztools.net/Health/Animal/Alternative_Medicine/",
                  "http://dmoztools.net/Health/Medicine/Evidence_Based_Medicine/"
                 ]
    itemlist = []

    @classmethod
    def from_crawler(cls, crawler):
        spider = super().from_crawler(crawler)
        crawler.signals.connect(spider.spider_closed, signals.spider_closed)
        return spider

    
    #csv writer
    def spider_closed(self):
        with open("dmoz.csv","w", newline="") as f:
            writer = csv.DictWriter(f,['Category','WebsiteName','WebsiteURL','WebpageTitle'])
            writer.writeheader()
            for data in self.itemlist:
                writer.writerow(data)

                
    # parser
    def parse(self, response):
        for record in response.css('.site-item'):
            items = {}
            items["Category"] = response.xpath('//title/text()').get()
            items["WebsiteName"] = record.css('.site-title::text').extract_first(default='')
            items["WebsiteURL"] = record.css('.title-and-desc a::attr(href)').extract_first(default='')
            
            if items["WebsiteURL"]:
                
                request = response.follow(items["WebsiteURL"], callback=self.get_website_title)
                request.meta['items'] = items
                yield request
            self.itemlist.append(items)
            
    # parsing child page contents
    def get_website_title(self,response):
        items = response.meta['items']
        items['WebpageTitle']=response.xpath('//title/text()').extract_first(default='').strip()
        yield items
        
        

cat_spider = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0',   
})
cat_spider.crawl(CatalogSpider)
cat_spider.start()

:0: UserWarning: You do not have a working installation of the service_identity module: 'cannot import name 'opentype''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.
2020-04-04 15:04:18 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-04-04 15:04:18 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.5.2 (default, Oct  8 2019, 13:06:37) - [GCC 5.4.0 20160609], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.4.0-170-generic-x86_64-with-Ubuntu-16.04-xenial
2020-04-04 15:04:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-04-04 15:04:18 [scrapy.crawler] INFO: Overridden

2020-04-04 15:04:20 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.pethealthproducts.com.au/admin/index.php>
{'Category': 'DMOZ - Health: Animal: Drugs and Medications', 'WebsiteName': 'Pet Health Products ', 'WebpageTitle': 'Admin Area', 'WebsiteURL': 'http://www.pethealthproducts.com.au/'}
2020-04-04 15:04:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.avidid.com/> (referer: http://dmoztools.net/Health/Animal/Drugs_and_Medications/)
2020-04-04 15:04:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.zoetisus.com/index.aspx> (referer: http://dmoztools.net/Health/Animal/Drugs_and_Medications/)
2020-04-04 15:04:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.zoetisus.com/Pages/inovocox2/inovocox2.aspx> (referer: http://dmoztools.net/Health/Animal/Drugs_and_Medications/)
2020-04-04 15:04:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.zoetisus.com/products/dairy/lutalyse_injection.aspx> (referer: http://dmoztools.net/Health

2020-04-04 15:04:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://bestpractice.bmj.com/info/evidence-information/> (referer: http://dmoztools.net/Health/Medicine/Evidence_Based_Medicine/)
2020-04-04 15:04:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.touchstone-systems.co.uk/> (referer: http://dmoztools.net/Computers/Software/Databases/Data_Warehousing/Consultants/)
2020-04-04 15:04:21 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.sisg.com/>
{'Category': 'DMOZ - Computers: Software: Databases: Data Warehousing: Consultants', 'WebsiteName': 'Strategic Information Systems Group ', 'WebpageTitle': 'Strategic Information Systems Group - Data Integration - Data Warehousing - Business Intelligence - Data Conversion - Technical Architecture', 'WebsiteURL': 'http://www.sisg.com/'}
2020-04-04 15:04:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.merck-animal-health-usa.com/product/cattle/Nuflor-Injectable-Solution/1/>
{'Category': 'DMOZ - Health

2020-04-04 15:04:22 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET http://guidehouse.com/> from <GET https://www.navigant.com/>
2020-04-04 15:04:22 [scrapy.core.scraper] DEBUG: Scraped from <200 http://inteligencija.com/>
{'Category': 'DMOZ - Computers: Software: Databases: Data Warehousing: Consultants', 'WebsiteName': 'Poslovna Inteligencija ', 'WebpageTitle': 'Poslovna inteligencija -', 'WebsiteURL': 'http://www.inteligencija.com/'}
2020-04-04 15:04:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.trinus.com/>
{'Category': 'DMOZ - Computers: Software: Databases: Data Warehousing: Consultants', 'WebsiteName': 'Trinus Corporation ', 'WebpageTitle': 'Trinus Corporation - Engaging with insight.', 'WebsiteURL': 'http://www.trinus.com/'}
2020-04-04 15:04:22 [scrapy.core.downloader.tls] WARNING: Remote certificate is not valid for hostname "www.huronconsultinggroup.com"; '*.huronconsultinggroup.com'!='www.huronconsultinggroup.com'
2020-04-04 15:04:22 

2020-04-04 15:04:24 [scrapy.core.engine] DEBUG: Crawled (406) <GET http://www.hiredbrains.com/> (referer: http://dmoztools.net/Computers/Software/Databases/Data_Warehousing/Consultants/)
2020-04-04 15:04:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.myersholum.com/>
{'Category': 'DMOZ - Computers: Software: Databases: Data Warehousing: Consultants', 'WebsiteName': 'Myers-Holum, Inc. ', 'WebpageTitle': 'Myers-Holum', 'WebsiteURL': 'http://www.myersholum.com/'}
2020-04-04 15:04:24 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.full360.com/> from <GET http://www.full360.com/>
2020-04-04 15:04:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://guidehouse.com/> (referer: http://dmoztools.net/Computers/Software/Databases/Data_Warehousing/Consultants/)
2020-04-04 15:04:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://relavision.com/>
{'Category': 'DMOZ - Computers: Software: Databases: Data Warehousing: Consultants', 'Websi

...

2020-04-04 15:04:40 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.evidence.nhs.uk/>
{'Category': 'DMOZ - Health: Medicine: Evidence Based Medicine', 'WebsiteName': 'NHS Evidence Search ', 'WebpageTitle': 'Evidence search | NICE', 'WebsiteURL': 'http://www.evidence.nhs.uk/'}
2020-04-04 15:04:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://joannabriggs.org/> (referer: http://dmoztools.net/Health/Medicine/Evidence_Based_Medicine/)
2020-04-04 15:04:40 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://ktclearinghouse.ca/cebm/> from <GET http://ktclearinghouse.ca/cebm/>
2020-04-04 15:04:40 [scrapy.core.scraper] DEBUG: Scraped from <200 https://guides.lib.unc.edu/c.php?g=8472&p=43571>
{'Category': 'DMOZ - Health: Medicine: Evidence Based Medicine', 'WebsiteName': 'Outcomes & Evidence Based Medicine ', 'WebpageTitle': 'Resources - EBP Education Center of Excellence - LibGuides at University of North Carolina at Chapel Hill', 'WebsiteURL': '

2020-04-04 15:04:42 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.hayesinc.com/hayes/>
{'Category': 'DMOZ - Health: Medicine: Evidence Based Medicine', 'WebsiteName': 'Hayes Health Technology Assessment ', 'WebpageTitle': 'Hayes, Inc.', 'WebsiteURL': 'http://www.hayesinc.com/'}
2020-04-04 15:04:42 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://ebm.jamanetwork.com/> from <GET http://ebm.jamanetwork.com/>
2020-04-04 15:04:42 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (302) to <GET https://onlinelibrary.wiley.com/doi/abs/10.1111/jpc.12800> from <GET https://onlinelibrary.wiley.com/doi/full/10.1111/jpc.12800>
2020-04-04 15:04:42 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (302) to <GET https://www.evidencealerts.com/> from <GET http://plus.mcmaster.ca/evidenceUpdates/>
2020-04-04 15:04:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.systematic-reviews.com/> (referer: http://dmoztools.net/Health/Medic

2020-04-04 15:04:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gpnotebook.com/simplepage.cfm?ID=-1596981199> (referer: http://dmoztools.net/Health/Medicine/Evidence_Based_Medicine/)
2020-04-04 15:04:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.evicom.co.uk/> (referer: http://dmoztools.net/Health/Medicine/Evidence_Based_Medicine/)
2020-04-04 15:04:43 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://bestbets.org/> from <GET http://www.bestbets.org/>
2020-04-04 15:04:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gpnotebook.com/simplepage.cfm?ID=-1596981199>
{'Category': 'DMOZ - Health: Medicine: Evidence Based Medicine', 'WebsiteName': 'General Practice Notebook: Evidence-Based Medicine ', 'WebpageTitle': 'evidence-based medicine - General Practice Notebook', 'WebsiteURL': 'http://www.gpnotebook.co.uk/simplepage.cfm?ID=-1596981199'}
2020-04-04 15:04:43 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) t

2020-04-04 15:05:18 [scrapy.extensions.logstats] INFO: Crawled 177 pages (at 177 pages/min), scraped 151 items (at 151 items/min)
2020-04-04 15:06:18 [scrapy.extensions.logstats] INFO: Crawled 177 pages (at 0 pages/min), scraped 151 items (at 0 items/min)
2020-04-04 15:06:27 [scrapy.downloadermiddlewares.retry] DEBUG: Retrying <GET http://www.thotwave.com/> (failed 1 times): TCP connection timed out: 110: Connection timed out.
2020-04-04 15:06:29 [scrapy.downloadermiddlewares.retry] DEBUG: Retrying <GET http://www.knightsbridge.com/> (failed 1 times): TCP connection timed out: 110: Connection timed out.
2020-04-04 15:06:31 [scrapy.downloadermiddlewares.retry] DEBUG: Retrying <GET http://pets.groups.yahoo.com/group/vetnet/> (failed 1 times): TCP connection timed out: 110: Connection timed out.
2020-04-04 15:06:31 [scrapy.downloadermiddlewares.retry] DEBUG: Retrying <GET http://tech.groups.yahoo.com/group/wwvffa/> (failed 1 times): TCP connection timed out: 110: Connection timed out.
202

2020-04-04 15:11:03 [scrapy.core.engine] INFO: Spider closed (finished)
